In [1]:
from pyspark import SparkConf, SparkContext

1. Creamos un contexto nuevo de Spark para leer un archivo de texto

In [2]:
conf = SparkConf().setAppName('readFile')
spark = SparkContext.getOrCreate(conf=conf)

your 131072x1 screen size is bogus. expect trouble


24/02/29 21:19:10 WARN Utils: Your hostname, DESKTOP-SLEQT56 resolves to a loopback address: 127.0.1.1; using 172.25.13.138 instead (on interface eth0)
24/02/29 21:19:10 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


24/02/29 21:19:12 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [110]:
text = spark.textFile("/mnt/d/Proyectos/Tutorial-SparkAWS/data/sampletext.txt")

In [111]:
type(text)

pyspark.rdd.RDD

In [112]:
text.collect()

['1 2 23 4', '10 23 101 67', '45 65 34 56', '101 54 34 101']

2. Sobre el __rdd__ aplicamos una __transformacion__ map que devuelve otro __rdd__

In [113]:
miFile_rdd = text.map(lambda x:x.split(' '))


In [114]:
type(miFile_rdd)

pyspark.rdd.PipelinedRDD

In [115]:
miFile_rdd.collect()

[['1', '2', '23', '4'],
 ['10', '23', '101', '67'],
 ['45', '65', '34', '56'],
 ['101', '54', '34', '101']]

3. En este ejemplo queremos usar una funcion __foo__ definida por el usuario donde, primero hagamos lo mismo que se hizo en __2__ pero luego se castea cada elemento a __int__ y se le suma 2.

In [116]:
def mi_foo(x):
    l =  x.split(' ')
    ls = []
    for num in l:
        ls.append(int(num) + 2)
    return ls

miFile  = text.map(mi_foo)
miFile.collect()

[[3, 4, 25, 6], [12, 25, 103, 69], [47, 67, 36, 58], [103, 56, 36, 103]]

Ejercicio.

Tenemos un archivo con saludos (texto) y debemos devolver un array con la logitud de cada palabra.

In [118]:
fileText = spark.textFile("/mnt/d/Proyectos/Tutorial-SparkAWS/data/wordcount.txt")

In [119]:
fileText.collect()

['Hi how are you?', 'Hope you are doing', 'great']

In [120]:
def foo(x):
    array = x.split(' ')
    l2 = []
    for word in array:
        l2.append(len(word))
    return l2

fileText.map(foo).collect()

[[2, 3, 3, 4], [4, 3, 3, 5], [5]]

Variacion del ejercicio.

En lugar de usar una __UDF__ usamos una funcion __lambda__ con una __list comprehention__ para obtener el mismo resultado.

In [121]:
fileText.map(lambda x: [len(word) for word in x.split(' ')]).collect()

[[2, 3, 3, 4], [4, 3, 3, 5], [5]]

4. FlatMap

Devuelve todos los elementos en un tipo de dato __unificado__  generando un nuevo __rdd__

In [122]:
fileText.flatMap(lambda x:x.split(' ')).collect()

['Hi', 'how', 'are', 'you?', 'Hope', 'you', 'are', 'doing', 'great']

5. filter - lambda

Se usa para segmentar el resultado de una __rdd__

In [123]:
text.filter(lambda x: x in ('10 23 45 67', '87 54 34 101')).collect()

[]

Usar filter sin __lambda function__

In [124]:
def foo(x):
    if x in ('10 23 45 67', '87 54 34 101'):
        return True
    return False


text.filter(foo).collect()

[]

6.1 Ejercicio 2

Construir un filtro que elimine las palabras que comiencen con __a__ o __c__

In [125]:
fileText = spark.textFile("/mnt/d/Proyectos/Tutorial-SparkAWS/data/quiz2.txt")
fileText.collect()

['this mango company animal',
 'cat dog ant mic laptop',
 'chair seitch mobile am charger cover',
 'amanda any alarm ant',
 'este es mi ejemplo']

Con lambda

In [126]:
rdd_rts = fileText.flatMap(lambda x:x.split(' '))\
    .filter(lambda x: x.startswith("a") or x.startswith("c"))
print(rdd_rts.collect())

['company', 'animal', 'cat', 'ant', 'chair', 'am', 'charger', 'cover', 'amanda', 'any', 'alarm', 'ant']


Sin lambda

In [127]:
def foo(x):
    if x.startswith("a") or x.startswith("c"):
        return False
    else:
        return True
    
    
rdd_rts = fileText.flatMap(lambda x:x.split(' ')).filter(foo)
print(rdd_rts.collect())  

['this', 'mango', 'dog', 'mic', 'laptop', 'seitch', 'mobile', 'este', 'es', 'mi', 'ejemplo']


7. RDD Distinct

In [128]:
rdd_text = text.flatMap(lambda x:x.split(' ')).distinct()
print(rdd_text.collect())

['1', '4', '10', '45', '34', '56', '54', '2', '23', '101', '67', '65']


In [129]:
text.map(lambda x: x.split(' ')).collect()

[['1', '2', '23', '4'],
 ['10', '23', '101', '67'],
 ['45', '65', '34', '56'],
 ['101', '54', '34', '101']]

8. GroupByKey

In [138]:
rdd_split = text.flatMap(lambda x: x.split(' '))
rdd_key_val = rdd_split.map(lambda x: (x, 1))
rdd_reduce = rdd_key_val.groupByKey()
rdd_mapVal = rdd_reduce.mapValues(list).collect()
print(rdd_mapVal)

[('1', [1]), ('4', [1]), ('10', [1]), ('45', [1]), ('34', [1, 1]), ('56', [1]), ('54', [1]), ('2', [1]), ('23', [1, 1]), ('101', [1, 1, 1]), ('67', [1]), ('65', [1])]


9. reduceByKey

Similar al GroupByKey pero devuelve un __rdd__ agrupado con una __operacion__ sobre los valores de las __keys__

In [144]:
rdd_split = text.flatMap(lambda x:x.split(' '))
rdd_map = rdd_split.map(lambda x: (x, 1))
rdd_reduce = rdd_map.reduceByKey(lambda x,y: x + y)
print(rdd_reduce.collect())

[('1', 1), ('4', 1), ('10', 1), ('45', 1), ('34', 2), ('56', 1), ('54', 1), ('2', 1), ('23', 2), ('101', 3), ('67', 1), ('65', 1)]


10. Ejercicio 

Escribir un script que devuelva la cantidad de cada palabra contenida en el archivo.

In [145]:
from pyspark import SparkContext, SparkConf

In [157]:
textFile = spark.textFile("/mnt/d/Proyectos/Tutorial-SparkAWS/data/quiz2.txt")
type(textFile)

#textFile.collect()
rdd_flat= textFile.flatMap(lambda x:x.split(' '))
rdd_key_val = rdd_flat.map(lambda x: (x, 1))
rdd_reduce = rdd_key_val.reduceByKey(lambda x, y: x + y)
print(rdd_reduce.collect())

[('this', 1), ('mango', 1), ('cat', 1), ('ant', 2), ('laptop', 1), ('chair', 1), ('mobile', 1), ('am', 1), ('este', 1), ('mi', 1), ('ejemplo', 1), ('company', 1), ('animal', 1), ('dog', 1), ('mic', 1), ('seitch', 1), ('charger', 1), ('cover', 1), ('amanda', 1), ('any', 1), ('alarm', 1), ('es', 1)]


11. Count y CountByValue

Ambas son acciones que se aplican sobre __RDD__
Count devuelve la cantidad de elementos dentro de un __rdd__
CountByValue devuelve la cantidad de veces que una elemento aparece en un __rdd__

In [171]:
print(textFile.count())
print(rdd_key_val.count())
print(rdd_flat.countByValue())


5
23
defaultdict(<class 'int'>, {'this': 1, 'mango': 1, 'company': 1, 'animal': 1, 'cat': 1, 'dog': 1, 'ant': 2, 'mic': 1, 'laptop': 1, 'chair': 1, 'seitch': 1, 'mobile': 1, 'am': 1, 'charger': 1, 'cover': 1, 'amanda': 1, 'any': 1, 'alarm': 1, 'este': 1, 'es': 1, 'mi': 1, 'ejemplo': 1})


In [176]:
for key, value in rdd_flat.countByValue().items():
    if value > 1:
        print (key, ':', value)

ant : 2


12. Rdd saveAsTextFile

Se usa para guardar un __rdd__ como un archivo de texto. POr defecto un __rdd__ está compuesto por dos particiones, asique __vamos a ver dos archivos__ con el 50% de los datos cada uno

In [179]:
print(rdd_flat.getNumPartitions())
rdd_flat.saveAsTextFile("../data/miRDDTextFile")

13. Aumentar numero de particiones

Se puede usar __repartition__ para aumentar o __coalesce__ para disminuirlo.

In [187]:
textFile = spark.textFile("/mnt/d/Proyectos/Tutorial-SparkAWS/data/quiz2.txt")


rdd_flat = textFile.flatMap(lambda x:x.split(' '))
print(rdd_flat.getNumPartitions())

rdd_flat_4 = rdd_flat.repartition(4)
rdd_flat_4.saveAsTextFile("../data/miPartition")
print(rdd_flat_4.getNumPartitions())

2


4


EN el ejemplo anterior vemos que se generan 4 archivos de particiones, ahora podemos usar __coalesce__ para que nos quede solo una particion.

In [188]:
print(rdd_flat_4.getNumPartitions())
rdd_flat_coalesce = rdd_flat_4.coalesce(1)
rdd_flat_coalesce.saveAsTextFile("../data/miPartitionColasce")
print(rdd_flat_coalesce.getNumPartitions())

4
1


14. Finding AVG

Para esta practica vamos a trabajar con el dataset __movie_ratings.csv__

In [10]:
movieRDD = spark.textFile("/mnt/d/Proyectos/Tutorial-SparkAWS/data/movie_ratings.csv")
print(movieRDD.count())

20


1. Vamos a crear un nuevo RDD que tenga las __tuplas__ con (key, (val,1))donde key= pelicula y val= raiting y 1 es un numero constante que luego vamos a sumar para nos devuelva la __cantidad de ocurrencias__

In [62]:
movieRDD_map = movieRDD.map(lambda x: (\
                                        x.split(",")[0], \
                                        (int(x.split(",")[1]),1)
                                        )
                            )


[('The Shawshank Redemption', (3, 1)), ('The Matrix', (5, 1)), ('12 Angry Men', (3, 1)), ('12 Angry Men', (4, 1)), ('The Matrix', (5, 1)), ('Pulp Fiction', (4, 1)), ('The Godfather', (5, 1)), ('The Shawshank Redemption', (2, 1)), ('Pulp Fiction', (4, 1)), ('The Godfather', (5, 1)), ('12 Angry Men', (2, 1)), ('The Godfather', (3, 1)), ('Pulp Fiction', (4, 1)), ('12 Angry Men', (1, 1)), ('The Shawshank Redemption', (2, 1)), ('12 Angry Men', (1, 1)), ('The Shawshank Redemption', (5, 1)), ('Pulp Fiction', (5, 1)), ('Pulp Fiction', (2, 1)), ('The Matrix', (4, 1))]


2. Luego reducimos con un __.reduceByKey__ para obtener algo así:
(key, (suma(val), suma(1))) o lo que seria igual a 
(key, (x[0]+y[0], x[1]+y[1]))

In [71]:
movieRDD_reduce = movieRDD_map.reduceByKey(lambda x, y: (x[0]+y[0], x[1]+y[1]))
print(movieRDD_reduce.collect())

[('The Shawshank Redemption', (12, 4)), ('12 Angry Men', (11, 5)), ('The Godfather', (13, 3)), ('The Matrix', (14, 3)), ('Pulp Fiction', (19, 5))]


Lo que necesitamos hacer es una __lambda__ que reciba x y use x[0] para la __key__ x[1][0] para __la suma de los raitings__ y x[1][1] __para la cantidad de ocurrencias__ y calcule el promedio

In [72]:
movieRDD_reduce.map(lambda x:(x[0], x[1][0]/x[1][1])).collect()

[('The Shawshank Redemption', 3.0),
 ('12 Angry Men', 2.2),
 ('The Godfather', 4.333333333333333),
 ('The Matrix', 4.666666666666667),
 ('Pulp Fiction', 3.8)]

15. Ejercicio rapido calcula AVG por mes

In [3]:
mesRDD = spark.textFile("/mnt/d/Proyectos/Tutorial-SparkAWS/data/average_quiz_sample.csv")
print(mesRDD.count())

12


Este problema se resuelve como el ejemplo anterior de __AVG__ pero una vez que tenemos el dataset debemos eliminar la variable __city__ ya que solo nos importa __mes__ y __raiting__

In [4]:
mesRDD_map = mesRDD.map(lambda x: x.split(","))

mesRDD_map2 = mesRDD_map.map(lambda x: (x[0], (int(float(x[2])), 1)))

mesRDD_map2_reduce = mesRDD_map2.reduceByKey(lambda x, y: (x[0]+y[0], x[1] +y[1]) )

mesRDD_map2_reduce.map(lambda x:(x[0], x[1][0] / x[1][1])).collect()

[('JAN', 2.5), ('FEB', 1.25), ('MAR', 2.0)]

16. Maximos y minimos en RDD

Buscamos para cada pelicula el __maximo y minimo__ raiting para cada pelicula

Para poder calcular Maximo y Minimo usamos __reduceByKey()__ donde esta transformacion toma los grupos de (key, val) y lo que queremos es que la __lambda x,y:  devuelva si x > y__

Es importante destacar que esta comparación en una lambda devuelve _un booleano__ por __true o false__ así que hay que especificarle lo que queremos ver.

1. Creamos el __RDD__ con (Key, Val)

In [18]:
movieRDD = spark.textFile("/mnt/d/Proyectos/Tutorial-SparkAWS/data/movie_ratings.csv")
print(movieRDD.count())

movieRDD_map = movieRDD.map(lambda x:(x.split(",")[0], int(x.split(",")[1]) ))
print(movieRDD_map.collect())

20
[('The Shawshank Redemption', 3), ('The Matrix', 5), ('12 Angry Men', 3), ('12 Angry Men', 4), ('The Matrix', 5), ('Pulp Fiction', 4), ('The Godfather', 5), ('The Shawshank Redemption', 2), ('Pulp Fiction', 4), ('The Godfather', 5), ('12 Angry Men', 2), ('The Godfather', 3), ('Pulp Fiction', 4), ('12 Angry Men', 1), ('The Shawshank Redemption', 2), ('12 Angry Men', 1), ('The Shawshank Redemption', 5), ('Pulp Fiction', 5), ('Pulp Fiction', 2), ('The Matrix', 4)]


2. Usando __.reduceByKey()__ traemos el minimo por Pelicula


In [16]:
movieRDD_map.reduceByKey(lambda x, y: x  if x > y else y).collect()

[('The Shawshank Redemption', 5),
 ('12 Angry Men', 4),
 ('The Godfather', 5),
 ('The Matrix', 5),
 ('Pulp Fiction', 5)]

3. Usando __.reduceByKey()__ traemos el maximo por Pelicula


In [17]:
movieRDD_map.reduceByKey(lambda x, y: x  if x < y else y).collect()

[('The Shawshank Redemption', 2),
 ('12 Angry Men', 1),
 ('The Godfather', 3),
 ('The Matrix', 4),
 ('Pulp Fiction', 2)]

17. Ejercicio rapido Maximos y Minimos.

Usando el dataset __average_quiz_samples.csv__ mostrar para cada ciudad el maximo y minimo raiting.

In [6]:
cityRDD = spark.textFile("/mnt/d/Proyectos/Tutorial-SparkAWS/data/average_quiz_sample.csv")
print(cityRDD.count())

12


1. Creamos el par (key, val)

In [7]:
cityRDD_map = cityRDD.map(lambda x: (x.split(",")[1], int(float(x.split(",")[2]))))


2. Buscamos el Maximo usando __lambda__

In [8]:
cityRDD_max = cityRDD_map.reduceByKey(lambda x,y: x if x > y else y)
print(cityRDD_max.collect())

[('NY', 3), ('CT', 4), ('PA', 3), ('NJ', 2), ('VT', 2)]


3. Buscamos el Minimo usando __lambda__

In [9]:
cityRDD_min = cityRDD_map.reduceByKey(lambda x,y: x if x < y else y)
print(cityRDD_min.collect())

[('NY', 1), ('CT', 4), ('PA', 1), ('NJ', 1), ('VT', 1)]


18. Proyecto integrador

Para este proyecto se pide usar el dataset __StudentData.csv__ y contestar las preguntas que estan en el __readme.md__

In [3]:
Students = spark.textFile("/mnt/d/Proyectos/Tutorial-SparkAWS/data/StudentData.csv")
print(Students.count())
#print(Students.collect())

1001


+ Elminamos el Header y nos quedamos solo con los datos

In [4]:
header = Students.first()
Students_one = Students.filter(lambda x:x != header)

1. El numero de estudiantes en el File.


In [5]:
Students_one.count()

1000

2. El total de notas por estudiantes M y F.


In [6]:
Students_MF_RDD = Students_one.map(lambda x: \
    (
        int(x.split(",")[0]),
        x.split(',')[1],
        x.split(',')[2],
        x.split(',')[3],
        x.split(',')[4],
        int(x.split(',')[5]),
        x.split(',')[6]
    ))

##Students_MF_RDD.collect()

3. Mostrar el total de alumnos que aprobaron o desaprobaron. Para aprobar se necesita 50+ puntos.


In [7]:
Students_MF_RDD.map(lambda x: (x[1],x[5])).reduceByKey(lambda x, y: x + y).collect()

[('Female', 29636), ('Male', 30461)]

3. Mostrar el total de alumnos que aprobaron o desaprobaron. Para aprobar se necesita 50+ puntos.


Hay dos formas de obtener el numero de estudiantes que fallaron.
1. Aplicando un filtro.
2. Obtenemos uno de los dos y se lo restamos al total

In [8]:
rdd_pass = Students_MF_RDD.filter(lambda x: x[5]> 50).count()
rdd_fail = Students_MF_RDD.filter(lambda x: x[5] <= 50).count()
print(rdd_fail, rdd_pass)

370 630


4. El numero total de alumnos por curso.


In [9]:
Students_MF_RDD.map(lambda x: (x[3], 1)).reduceByKey(lambda x,y: x + y).collect()

[('DB', 157),
 ('Cloud', 192),
 ('PF', 166),
 ('MVC', 157),
 ('OOP', 152),
 ('DSA', 176)]

5. El total de notas por curso.


In [10]:
Students_MF_RDD.map(lambda x: (x[3], x[5])).reduceByKey(lambda x,y: x+y).collect()

[('DB', 9270),
 ('Cloud', 11443),
 ('PF', 9933),
 ('MVC', 9585),
 ('OOP', 8916),
 ('DSA', 10950)]

6. El AVG de notas de alumnos por curso.

+ Usando __map__

In [11]:
Students_MF_RDD.map(lambda x: (x[3], (x[5], 1)))\
    .reduceByKey(lambda x,y : (x[0] + y[0], x[1] + y[1]))\
        .map(lambda x: (x[0], x[1][0]/ x[1][1]))\
            .collect()

[('DB', 59.044585987261144),
 ('Cloud', 59.598958333333336),
 ('PF', 59.83734939759036),
 ('MVC', 61.05095541401274),
 ('OOP', 58.6578947368421),
 ('DSA', 62.21590909090909)]

Usando __mapValue__

In [18]:
Students_MF_RDD.map(lambda x: (x[3], (x[5], 1)))\
    .reduceByKey(lambda x,y : (x[0] + y[0], x[1] + y[1]))\
        .mapValues(lambda x: (x[0] /  x[1]))\
            .collect()

[('DB', 59.044585987261144),
 ('Cloud', 59.598958333333336),
 ('PF', 59.83734939759036),
 ('MVC', 61.05095541401274),
 ('OOP', 58.6578947368421),
 ('DSA', 62.21590909090909)]

Otra forma es usando __mapValues()__ que por cada __key__ devuelve los valores, en este caso la __tupla__ haciendo que podamos trabajar directamente con ella.

7. El maximo y minimo de notas por curso.


In [14]:
Students_MF_RDD.map(lambda x: (x[3], x[5])).reduceByKey(lambda x,y: x if x > y else y).collect()

[('DB', 98), ('Cloud', 99), ('PF', 99), ('MVC', 99), ('OOP', 99), ('DSA', 99)]

In [15]:
Students_MF_RDD.map(lambda x: (x[3], x[5])).reduceByKey(lambda x,y: x if x < y else y).collect()

[('DB', 20), ('Cloud', 20), ('PF', 20), ('MVC', 22), ('OOP', 20), ('DSA', 20)]

8. EL AVG de edad de M y F.


In [23]:
Students_MF_RDD.map(lambda x: (x[1], (x[0], 1)))\
    .reduceByKey(lambda x,y:(x[0]+y[0], x[1]+y[1]))\
        .mapValues(lambda x: x[0] / x[1] ).collect()

[('Female', 28.489021956087825), ('Male', 28.52304609218437)]

Para resolver este ultimo problema usamos __mapValue__ que lo aprendimos en el ejemplo anterior pero tambien podriamos usar __map__ y usar los indices y subindices de la __tupla__